# Setup

In [59]:
from sklearn.datasets import load_digits
from sklearn.datasets import fetch_openml

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

    
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [60]:
#mnist = load_digits()
mnist = fetch_openml('mnist_784', version=1, as_frame=False)

# 1. EDA

# 2.Prep

## Split

In [72]:
X, y = mnist["data"], mnist["target"].astype(np.uint8)

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

## Dimensionality Reduction

### 1.PCA

### 2.t-SNE

## Scaling

In [65]:
# Digit
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")

# plot_digit(X[0])
    
    
    
# Digits
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.figure(figsize=(9,9))
    plt.imshow(image, cmap = mpl.cm.binary, **options)
    plt.axis("off")
    
# plot_digits(X[:100]) 

# 2.Classification

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score

Algorithms
    Binary
        SGDClassifier
        RandonForestClassifier
    Multiclass
        svm (C-Support Vector Classifier)
        OneVsRestClassifier

## 1.Binary classifiers

In [74]:
# 5 against the rest
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

## 1.1.SGD Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

In [79]:
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

SGDClassifier(random_state=42)

In [80]:
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

## 1.2.Random Forest Classifier

In [106]:
from sklearn.ensemble import RandomForestClassifier

In [94]:
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)

y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3,
                                    method="predict_proba")

In [ ]:
y_train_pred_forest = cross_val_predict(forest_clf, X_train, y_train_5, cv=3)
precision_score(y_train_5, y_train_pred_forest)

In [96]:
y_scores_forest = y_probas_forest[:, 1] # score = proba of positive class
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5,y_scores_forest)

## 2.Multiclass classifiers

##  2.1.SVC

In [ ]:
from sklearn.svm import SVC

In [101]:
svm_clf = SVC(gamma="auto", random_state=42)
svm_clf.fit(X_train[:1000], y_train[:1000]) # y_train, not y_train_5
svm_clf.predict([some_digit])

array([5], dtype=uint8)

## 2.2 One vs Rest 

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
ovr_clf = OneVsRestClassifier(SVC(gamma="auto", random_state=42))
ovr_clf.fit(X_train[:1000], y_train[:1000])
ovr_clf.predict([some_digit])

# 4.Evaluation

### Metrics  
    1.Precision  
    2.Recall  
    3.f1_score  
### Curves  
    4.Precision vs Recall  
    5.Precision-Recall vs Threshold  
    6.ROC  
    7.ROC vs AUC  

In [1]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score,\
                            f1_score, precision_recall_curvebb, roc_curve, \
                            roc_auc_score

ImportError: cannot import name 'precision_recall_curvebb' from 'sklearn.metrics' (C:\Users\Amir\Anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

In [ ]:
def scores(pr,re,f1):
    print (f'Precision: {pr}' , '\n' + f'Recall: {re}', '\n' + f'f1: {f1}')


def plot_precision_vs_recall(precisions, recalls):
    plt.figure(figsize=(8, 6))
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])
    plt.grid(True)
    

def plot_roc_curve(fpr, tpr, label=None):
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16)
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)
    plt.grid(True)
    
    

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.figure(figsize=(8, 4))                                              
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16)
    plt.xlabel("Threshold", fontsize=16)
    plt.grid(True)
    plt.axis([-50000, 50000, 0, 1])

## SGDClassifier

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_train_5, y_train_pred)

### Precision, Recall, f1

In [2]:
pr = precision_score(y_train_5, y_train_pred)
#cm[1, 1] / (cm[0, 1] + cm[1, 1])

re = recall_score(y_train_5, y_train_pred)
#cm[1, 1] / (cm[1, 0] + cm[1, 1])

f1 = f1_score(y_train_5, y_train_pred)
# cm[1, 1] / (cm[1, 1] + (cm[1, 0] + cm[0, 1]) / 2)

scores(pr, re, f1)

NameError: name 'y_train_5' is not defined

### 4.Precision vs Recall  

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3,
                             method="decision_function")
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

plot_precision_vs_recall(precisions, recalls)
plt.plot([recall_90_precision, recall_90_precision], [0., 0.9], "r:")
plt.plot([0.0, recall_90_precision], [0.9, 0.9], "r:")
plt.plot([recall_90_precision], [0.9], "ro")
plt.show()

### 5.Precision-Recall vs Threshold  

In [ ]:
recall_90_precision = recalls[np.argmax(precisions >= 0.90)]
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.plot([threshold_90_precision, threshold_90_precision], [0., 0.9], "r:")
plt.plot([-50000, threshold_90_precision], [0.9, 0.9], "r:")
plt.plot([-50000, threshold_90_precision], [recall_90_precision, recall_90_precision], "r:")
plt.plot([threshold_90_precision], [0.9], "ro")
plt.plot([threshold_90_precision], [recall_90_precision], "ro")
plt.show()

### 6.ROC

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)

plot_roc_curve(fpr, tpr)
fpr_90 = fpr[np.argmax(tpr >= recall_90_precision)]
plt.plot([fpr_90, fpr_90], [0., recall_90_precision], "r:")
plt.plot([0.0, fpr_90], [recall_90_precision, recall_90_precision], "r:")
plt.plot([fpr_90], [recall_90_precision], "ro")
plt.show()

### 7.ROC vs AUC 

In [ ]:
roc_auc_score(y_train_5, y_scores)

## Random Forest Classifier

### Confusion Matrix

### Precision, Recall, f1

NameError: name 'y_train_5' is not defined

### 4.Precision vs Recall  

### 5.Precision-Recall vs Threshold  

### 6.ROC

### 7.ROC vs AUC 

# 5.Tuning